In [1]:
import pprint
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from json_shot_scraper import flatten_shot, flatten_goal, flatten_complete_pass, flatten_incomplete_pass, flatten_corner
from  player_scraper import flatten_player, flatten_sub
from dataframe_cleaner import (pass_to_shot, corner_to_shot, transpose_coordinates, coord_to_yards, 
                               shot_distance_angle, dummy_columns, drop_own_goals, goal_dummy)

In [2]:
from html_scraper import db
from mongo_to_db import game_to_cleaned_df, create_frame, create_master_df, create_master_player_min_df

In [3]:
pd.set_option('display.max_columns', 50)

In [146]:
games = db.games.find()

# Players DF

In [147]:
players_minutes_df = create_master_player_min_df(games)

In [148]:
players_minutes_df.head()

,game_id,name,player_id,position_id,squad_number,substitute,team_id,minutes_played
0,448548,Alexander Domínguez,19132.0,1,22.0,False,20,90.0
1,448548,Lucas Hoyos,27987.0,1,12.0,True,20,0.0
2,448548,Gastón Díaz,30896.0,2,24.0,False,20,90.0
3,448548,Joaquín Laso,94225.0,2,6.0,False,20,90.0
4,448548,Luis Abram,99779.0,2,29.0,False,20,90.0


# Game Events DF

In [7]:
games = db.games.find()

In [8]:
shots_df = create_master_df(games)

/Users/david/galvanize/super_liga_xg/mongo_to_db.py:69: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  final_df = pd.concat([attach_to_df, df], axis=0, ignore_index=True)
/Users/david/galvanize/super_liga_xg/dataframe_cleaner.py:133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  input_df['is_goal'] = input_df.loc[:, 'shot_type'].isin([9, 11, 12, 13]).astype(int)


In [9]:
shots_df.head()

,game_id,player_id,shot_coord_x1,shot_coord_x2,shot_coord_y1,shot_coord_y2,shot_coord_z1,shot_coord_z2,shot_id,shot_type,team_id,time_of_event(min),passed_from_id,pass_coord_x1,pass_coord_x2,pass_coord_y1,pass_coord_y2,pass_coord_z1,pass_coord_z2,corner_kick,shot_distance,shot_angle,assisted_shot,is_penalty_attempt,is_goal
0,448548,99779.0,6.67,0.00,1.51,-1.21,-1,0.83,22336297,11,20,1.033333,177285.0,0.89,12.44,29.36,-4.84,-1.0,-1.0,1.0,6.838786,12.755990,1,0,1.0
1,448548,60730.0,21.33,12.00,4.24,-0.91,-1,0.73,22336386,35,20,9.683333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,21.747333,11.242763,0,0,0.0
2,448548,177285.0,12.00,1.78,0.00,-3.03,-1,0.21,22336583,13,20,26.833333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,12.000000,0.000000,0,1,1.0
3,448548,185745.0,8.44,0.89,14.53,-6.66,-1,-1.00,22336723,33,20,38.883333,177285.0,22.67,6.22,3.93,19.67,-1.0,-1.0,0.0,16.803407,59.849102,1,0,0.0
4,448548,77892.0,22.22,0.44,13.92,-8.48,-1,-1.00,22336739,33,13,40.083333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,26.220122,32.065607,0,0,0.0


In [10]:
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from sklearn.ensemble import  RandomForestClassifier

In [40]:
possible_games = list(shots_df['game_id'].unique())
len(possible_games)

159

## manually creating train_test_splt by game_id

start: 159 games --> removed 64 (64/80 were unique) games to train (train model)

test_set = 95 games left --> random sample (40 games) --> predict 


In [41]:
# the indices 80 games to train random forest on
games_to_train_on = np.random.choice(159, 80)

In [42]:
games_to_sample = []
for i in games_to_train_on:
    games_to_sample.append(possible_games[i])

In [43]:
len(games_to_sample)

80

In [44]:
for game in games_to_sample:
    if game in possible_games:
        possible_games.remove(game)

### remove 60 games from possible games

In [45]:
len(possible_games)

95

In [48]:
#games to train rf on
len(games_to_sample)

80

In [49]:
rf_columns = ['player_id', 'shot_distance', 'shot_angle', 'assisted_shot', 'is_penalty_attempt']

In [50]:
shots_to_train_on = shots_df[shots_df['game_id'].isin(np.array(games_to_sample))].copy()

In [51]:
len(shots_to_train_on['game_id'].unique())

64

In [52]:
shots_to_train_on.head()

,game_id,player_id,shot_coord_x1,shot_coord_x2,shot_coord_y1,shot_coord_y2,shot_coord_z1,shot_coord_z2,shot_id,shot_type,team_id,time_of_event(min),passed_from_id,pass_coord_x1,pass_coord_x2,pass_coord_y1,pass_coord_y2,pass_coord_z1,pass_coord_z2,corner_kick,shot_distance,shot_angle,assisted_shot,is_penalty_attempt,is_goal
0,448548,99779.0,6.67,0.00,1.51,-1.21,-1,0.83,22336297,11,20,1.033333,177285.0,0.89,12.44,29.36,-4.84,-1.0,-1.0,1.0,6.838786,12.755990,1,0,1.0
1,448548,60730.0,21.33,12.00,4.24,-0.91,-1,0.73,22336386,35,20,9.683333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,21.747333,11.242763,0,0,0.0
2,448548,177285.0,12.00,1.78,0.00,-3.03,-1,0.21,22336583,13,20,26.833333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,12.000000,0.000000,0,1,1.0
3,448548,185745.0,8.44,0.89,14.53,-6.66,-1,-1.00,22336723,33,20,38.883333,177285.0,22.67,6.22,3.93,19.67,-1.0,-1.0,0.0,16.803407,59.849102,1,0,0.0
4,448548,77892.0,22.22,0.44,13.92,-8.48,-1,-1.00,22336739,33,13,40.083333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,26.220122,32.065607,0,0,0.0


In [53]:
train_data = shots_to_train_on[rf_columns]
train_y = shots_to_train_on['is_goal']
indices = shots_to_train_on.index.values

In [54]:
random_forest_model = RandomForestClassifier(n_estimators=300, max_depth=3, verbose=1)
random_forest_model.fit(train_data, train_y)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.3s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=3, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=None,
            oob_score=False, random_state=None, verbose=1,
            warm_start=False)

### sample test set by game_id

In [55]:
len(possible_games)

95

In [56]:
games_to_test_on = np.random.choice(95, 50)

In [57]:
games_to_predict = []
for i in games_to_test_on:
    games_to_predict.append(possible_games[i])

In [58]:
print(len(possible_games))
len(games_to_predict)

95


50

remove from that sample

In [61]:
for game in games_to_predict:
    if game in possible_games:
        possible_games.remove(game)

In [62]:
print(len(possible_games))
len(games_to_predict)

61


50

In [63]:
95 - 61

34

### 34 unique games removed from the possible_games

In [64]:
shots_to_predict = shots_df[shots_df['game_id'].isin(np.array(games_to_predict))].copy()

In [65]:
shots_to_predict.head()

,game_id,player_id,shot_coord_x1,shot_coord_x2,shot_coord_y1,shot_coord_y2,shot_coord_z1,shot_coord_z2,shot_id,shot_type,team_id,time_of_event(min),passed_from_id,pass_coord_x1,pass_coord_x2,pass_coord_y1,pass_coord_y2,pass_coord_z1,pass_coord_z2,corner_kick,shot_distance,shot_angle,assisted_shot,is_penalty_attempt,is_goal
56,448539,17266.0,22.22,0.44,3.93,11.20,-1,-1.00,22355164,33,869,7.650000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,22.564869,10.030043,0,0,0.0
57,448539,47913.0,9.78,6.67,10.90,5.45,-1,0.46,22355363,35,6,14.900000,57586.0,41.78,11.11,17.25,11.20,-1.0,-1.0,0.0,14.644398,48.100031,1,0,0.0
58,448539,1692.0,19.56,0.44,11.80,-0.61,-1,-1.00,22355398,33,6,16.216667,57586.0,44.89,25.78,23.61,23.61,-1.0,-1.0,0.0,22.843677,31.101404,1,0,0.0
59,448539,44918.0,19.11,0.00,-11.50,-12.71,-1,-1.00,22355730,33,869,24.816667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,22.303410,-31.038653,0,0,0.0
60,448539,98292.0,12.44,9.78,2.72,2.42,-1,0.14,22355759,35,869,25.600000,44918.0,25.78,10.67,-10.59,-10.29,-1.0,-1.0,0.0,12.733892,12.333593,1,0,0.0


In [66]:
len(shots_to_predict['game_id'].unique())

34

# Predicting the xG xA of 34 games

In [67]:
test_data = shots_to_predict[rf_columns]
test_y = shots_to_predict['is_goal']
indices1 = shots_to_predict.index.values

In [68]:
p_random_forest = random_forest_model.predict_proba(test_data)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.1s finished


In [73]:
df_xG = pd.DataFrame(test_data)
df_xG['is_goal'] = test_y
df_xG['xG'] = p_random_forest[:,1]

In [74]:
df_xG.head()

,player_id,shot_distance,shot_angle,assisted_shot,is_penalty_attempt,is_goal,prob_0,prob_1,xG
56,17266.0,22.564869,10.030043,0,0,0.0,0.925429,0.074571,0.074571
57,47913.0,14.644398,48.100031,1,0,0.0,0.937550,0.062450,0.062450
58,1692.0,22.843677,31.101404,1,0,0.0,0.921333,0.078667,0.078667
59,44918.0,22.303410,-31.038653,0,0,0.0,0.955819,0.044181,0.044181
60,98292.0,12.733892,12.333593,1,0,0.0,0.927847,0.072153,0.072153


In [75]:
df_xG['xA'] = df_xG['assisted_shot'] * df_xG['xG']

In [77]:
df_xG.head()

,player_id,shot_distance,shot_angle,assisted_shot,is_penalty_attempt,is_goal,prob_0,prob_1,xG,xA
56,17266.0,22.564869,10.030043,0,0,0.0,0.925429,0.074571,0.074571,0
57,47913.0,14.644398,48.100031,1,0,0.0,0.937550,0.062450,0.062450,0.0624496
58,1692.0,22.843677,31.101404,1,0,0.0,0.921333,0.078667,0.078667,0.0786667
59,44918.0,22.303410,-31.038653,0,0,0.0,0.955819,0.044181,0.044181,0
60,98292.0,12.733892,12.333593,1,0,0.0,0.927847,0.072153,0.072153,0.0721525


In [78]:
unique_players = df_xG['player_id'].unique()

In [138]:
contributions = []
for player in unique_players:
    xgsum = df_xG[df_xG['player_id'] == player]['xG'].sum()
    xasum = df_xG[df_xG['player_id'] == player]['xA'].sum()
    goals = df_xG[df_xG['player_id'] == player]['is_goal'].sum()
    contributions.append([player, xgsum, xasum, goals])

In [139]:
by_xG = sorted(contributions, key=lambda x: x[1], reverse=True)

In [140]:
contribution_df = pd.DataFrame(by_xG, columns=['player_id', 'total_xG', 'total_xA', 'goals'])

In [141]:
contribution_df['combined xG + xA'] = (contribution_df['total_xG'] + contribution_df['total_xA']).round(2)

In [142]:
contribution_df.head()

,player_id,total_xG,total_xA,goals,combined xG + xA
0,488.0,1.507333,0.699513,4.0,2.21
1,618.0,1.398849,0.255770,1.0,1.65
2,16081.0,1.372461,0.080025,1.0,1.45
3,39578.0,1.312285,0.983070,3.0,2.30
4,81291.0,1.012700,0.000000,2.0,1.01


### now want to find the total minutes from the games in 'games_to_predict' and find the xG in that many minutes

In [149]:
pred_min_df = players_minutes_df[players_minutes_df['game_id'].isin(np.array(games_to_predict))].copy()

In [151]:
pred_min_df.head()

,game_id,name,player_id,position_id,squad_number,substitute,team_id,minutes_played
152,448539,Sebastián Bértoli,46612.0,1,1.0,False,869,90.0
153,448539,Federico Costa,34646.0,1,21.0,True,869,0.0
154,448539,Lucas Ceballos,29319.0,2,3.0,False,869,90.0
155,448539,Matías Escudero,144877.0,2,6.0,False,869,90.0
156,448539,Renzo Vera,16278.0,2,33.0,False,869,90.0


In [152]:
players = pred_min_df['player_id'].unique()
player_minutes = []
for player in players:
    total_minutes = pred_min_df[pred_min_df['player_id'] == player]['minutes_played'].sum()
    name = pred_min_df[pred_min_df['player_id'] == player]['name'].iloc[0]
    player_minutes.append([player, total_minutes, name])

In [153]:
player_minutes_df = pd.DataFrame(player_minutes, columns=['player_id', 'total_minutes_played', 'player_name'])

In [154]:
player_minutes_df.head()

,player_id,total_minutes_played,player_name
0,46612.0,180.0,Sebastián Bértoli
1,34646.0,0.0,Federico Costa
2,29319.0,180.0,Lucas Ceballos
3,144877.0,90.0,Matías Escudero
4,16278.0,90.0,Renzo Vera


# Merge contribution_df with player_minutes_df

In [159]:
xg_min = pd.merge(contribution_df, player_minutes_df, on='player_id', how='outer')

In [161]:
xg_min.head()

,player_id,total_xG,total_xA,goals,combined xG + xA,total_minutes_played,player_name
0,488.0,1.507333,0.699513,4.0,2.21,252.550000,Lisandro López
1,618.0,1.398849,0.255770,1.0,1.65,164.333333,Mauro Zárate
2,16081.0,1.372461,0.080025,1.0,1.45,199.733333,Luis Rodríguez
3,39578.0,1.312285,0.983070,3.0,2.30,390.583333,Lucas Gamba
4,81291.0,1.012700,0.000000,2.0,1.01,146.383333,Diego Mendoza


In [162]:
columns = ['player_name', 'player_id', 'total_xG', 'total_xA', 'combined xG + xA', 'goals', 'total_minutes_played']

In [163]:
xg_final = xg_min[columns]

In [164]:
xg_final.head()

,player_name,player_id,total_xG,total_xA,combined xG + xA,goals,total_minutes_played
0,Lisandro López,488.0,1.507333,0.699513,2.21,4.0,252.550000
1,Mauro Zárate,618.0,1.398849,0.255770,1.65,1.0,164.333333
2,Luis Rodríguez,16081.0,1.372461,0.080025,1.45,1.0,199.733333
3,Lucas Gamba,39578.0,1.312285,0.983070,2.30,3.0,390.583333
4,Diego Mendoza,81291.0,1.012700,0.000000,1.01,2.0,146.383333


In [165]:
xg_final['xG+xA/90'] = xg_final['combined xG + xA'].copy() / (xg_final['total_minutes_played'] / 90)

/Users/david/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [168]:
xg_final[xg_final['xG+xA/90'] > 0.5]

,player_name,player_id,total_xG,total_xA,combined xG + xA,goals,total_minutes_played,xG+xA/90
0,Lisandro López,488.0,1.507333,0.699513,2.21,4.0,252.550000,0.787567
1,Mauro Zárate,618.0,1.398849,0.255770,1.65,1.0,164.333333,0.903651
2,Luis Rodríguez,16081.0,1.372461,0.080025,1.45,1.0,199.733333,0.653371
3,Lucas Gamba,39578.0,1.312285,0.983070,2.30,3.0,390.583333,0.529977
4,Diego Mendoza,81291.0,1.012700,0.000000,1.01,2.0,146.383333,0.620972
5,Mauro Matos,31807.0,1.010512,0.539284,1.55,2.0,171.750000,0.812227
6,Emmanuel Gigliotti,24256.0,0.996530,0.303808,1.30,3.0,213.050000,0.549167
7,Gonzalo Martínez,77919.0,0.909089,0.090122,1.00,1.0,72.833333,1.235698
8,Darío Cvitanich,101.0,0.880758,0.380135,1.26,1.0,180.000000,0.630000
9,Pablo Alvarenga,150346.0,0.840975,0.475642,1.32,1.0,96.566667,1.230238


In [169]:
xg_final[xg_final['xG+xA/90'] > 1]

,player_name,player_id,total_xG,total_xA,combined xG + xA,goals,total_minutes_played,xG+xA/90
7,Gonzalo Martínez,77919.0,0.909089,0.090122,1.00,1.0,72.833333,1.235698
9,Pablo Alvarenga,150346.0,0.840975,0.475642,1.32,1.0,96.566667,1.230238
48,Julián Carranza,193905.0,0.391490,0.174401,0.57,1.0,45.000000,1.140000
56,Rodrigo Gómez,86727.0,0.354284,0.217184,0.57,0.0,47.683333,1.075848
67,Carlos Luna,3116.0,0.308500,0.077689,0.39,1.0,32.133333,1.092324
224,Joel Soñora,101036.0,0.062589,0.062589,0.13,0.0,7.116667,1.644028
230,Julián López,209413.0,0.060344,0.060344,0.12,0.0,2.500000,4.320000


In [171]:
xg_final[(xg_final['xG+xA/90'] > xg_final['goals']) & (xg_final['goals'] > 0)]

,player_name,player_id,total_xG,total_xA,combined xG + xA,goals,total_minutes_played,xG+xA/90
7,Gonzalo Martínez,77919.0,0.909089,0.090122,1.00,1.0,72.833333,1.235698
9,Pablo Alvarenga,150346.0,0.840975,0.475642,1.32,1.0,96.566667,1.230238
48,Julián Carranza,193905.0,0.391490,0.174401,0.57,1.0,45.000000,1.140000
67,Carlos Luna,3116.0,0.308500,0.077689,0.39,1.0,32.133333,1.092324
